### requests 라이브러리

- 웹 페이지를 다운로드하기 위한 라이브러리
- https://2.python-requests.org/en/master/user/quickstart/

In [1]:
import requests

page = requests.get("https://dataquestio.github.io/web-scraping-pages/simple.html")
page

<Response [200]>

In [2]:
# status_code 의미
# 2xx: 성공
# 4xx: 클라이언트 에러
# 5xx: 서버 에러
page.status_code

200

In [3]:
# binary response content
page.content

b'<!DOCTYPE html>\n<html>\n    <head>\n        <title>A simple example page</title>\n    </head>\n    <body>\n        <p>Here is some simple content for this page.</p>\n    </body>\n</html>'

### BeautifulSoup 라이브러리

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


In [6]:
list(soup.children)

['html',
 '\n',
 <html>
 <head>
 <title>A simple example page</title>
 </head>
 <body>
 <p>Here is some simple content for this page.</p>
 </body>
 </html>]

In [8]:
# Doctype: 문서 타입에 대한 정보
# NavigableString: HTML 문서에서 찾아낸 텍스트
# Tag: 중첩(nested) 태그
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

In [11]:
# html 태그 안의 자식에 대한 결과
html = list(soup.children)[2]
list(html.children)

['\n',
 <head>
 <title>A simple example page</title>
 </head>,
 '\n',
 <body>
 <p>Here is some simple content for this page.</p>
 </body>,
 '\n']

In [12]:
# body 태그 안 자식에 대한 결과
body = list(html.children)[3]
list(body.children)

['\n', <p>Here is some simple content for this page.</p>, '\n']

In [15]:
# p 태그 안 텍스트
p = list(body.children)[1]
p.get_text()

'Here is some simple content for this page.'

### 태그명으로 태그 검색

In [16]:
soup = BeautifulSoup(page.content, 'html.parser')
soup.find_all('p')

[<p>Here is some simple content for this page.</p>]

In [19]:
# find_all은 list를 리턴함
# 따라서 첫 번째 텍스트를 얻기 위해 다음과 같이 함
soup.find_all('p')[0].get_text()

'Here is some simple content for this page.'

In [18]:
# 첫 번째 인스턴스만 찾기 원할 경우 find 이용
soup.find('p')

<p>Here is some simple content for this page.</p>

### class와 id로 태그 검색

```html
<html>
    <head>
        <title>A simple example page</title>
    </head>
    <body>
        <div>
            <p class="inner-text first-item" id="first">
                First paragraph.
            </p>
            <p class="inner-text">
                Second paragraph.
            </p>
        </div>
            <p class="outer-text first-item" id="second">
                <b>
                First outer paragraph.
                </b>
            </p>
            <p class="outer-text">
                <b>
                Second outer paragraph.
                </b>
            </p>
    </body>
</html>
```

In [20]:
page = requests.get("https://dataquestio.github.io/web-scraping-pages/ids_and_classes.html")
soup = BeautifulSoup(page.content, 'html.parser')
soup

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                First paragraph.
            </p>
<p class="inner-text">
                Second paragraph.
            </p>
</div>
<p class="outer-text first-item" id="second">
<b>
                First outer paragraph.
            </b>
</p>
<p class="outer-text">
<b>
                Second outer paragraph.
            </b>
</p>
</body>
</html>

In [21]:
soup.find_all('p', class_='outer-text')

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>,
 <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [22]:
soup.find_all(class_="outer-text")

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>,
 <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

### CSS 선택자
- p a — finds all a tags inside of a p tag.
- body p a — finds all a tags inside of a p tag inside of a body tag.
- html body — finds all body tags inside of an html tag.
- p.outer-text — finds all p tags with a class of outer-text.
- p#first — finds all p tags with an id of first.
- body p.outer-text — finds any p tags with a class of outer-text inside of a body tag.

In [23]:
soup.select("div p")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>,
 <p class="inner-text">
                 Second paragraph.
             </p>]

### 날씨 데이터 스크래핑
https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168
- 날씨 예보 웹 페이지 다운로드
- 웹 페이지 파싱하기 위한 BeautifulSoup 객체 생성
- id가 seven-day-forecast인 div 태그 찾고 이를 seven_day에 할당
- seven_day 내부에서 예보 아이템 찾기
- 첫 번째 예보 아이템 추출 및 출력

In [26]:
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Tonight: Mostly clear, with a low around 47. West wind 14 to 17 mph, with gusts as high as 23 mph. " class="forecast-icon" src="newimages/medium/nfew.png" title="Tonight: Mostly clear, with a low around 47. West wind 14 to 17 mph, with gusts as high as 23 mph. "/>
 </p>
 <p class="short-desc">
  Mostly Clear
 </p>
 <p class="temp temp-low">
  Low: 47 °F
 </p>
</div>


- tonight 변수 안의 정보 추출
- 예보 아이템 이름: Tonight
- 예보 설명: img 태그 안의 title 속성
- 짧은 예보 설명: 위 경우에는 Mostly Clear임
- 최저 기온: 위 경우에는 47도

In [27]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

Tonight
Mostly Clear
Low: 47 °F


In [28]:
img = tonight.find("img")
desc = img['title']
print(desc)

Tonight: Mostly clear, with a low around 47. West wind 14 to 17 mph, with gusts as high as 23 mph. 


모든 정보 추출하기
- 변수 seven_day에서 class가 tombstone-container인 아이템 내부에서 class가 period-name인 모든 아이템을 선택
- get_text 메소드 호출하여 리스트 생성

In [30]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Tonight',
 'Friday',
 'FridayNight',
 'Saturday',
 'SaturdayNight',
 'Sunday',
 'SundayNight',
 'Monday',
 'MondayNight']

In [32]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['Mostly Clear', 'Mostly Sunny', 'Mostly Cloudyand Breezythen ChanceRain', 'Rain thenShowersLikely', 'Mostly Clear', 'Sunny', 'Mostly Clear', 'Sunny', 'Clear']
['Low: 47 °F', 'High: 63 °F', 'Low: 50 °F', 'High: 59 °F', 'Low: 45 °F', 'High: 63 °F', 'Low: 47 °F', 'High: 69 °F', 'Low: 53 °F']
['Tonight: Mostly clear, with a low around 47. West wind 14 to 17 mph, with gusts as high as 23 mph. ', 'Friday: Mostly sunny, with a high near 63. West wind 5 to 10 mph increasing to 12 to 17 mph in the afternoon. Winds could gust as high as 22 mph. ', 'Friday Night: A 50 percent chance of rain after midnight.  Increasing clouds, with a low around 50. Breezy, with a west wind 17 to 22 mph decreasing to 9 to 14 mph after midnight. Winds could gust as high as 29 mph.  New precipitation amounts of less than a tenth of an inch possible. ', 'Saturday: Rain before noon, then showers likely after noon.  High near 59. South southwest wind 7 to 11 mph becoming northwest in the afternoon.  Chance of precipita

모든 데이터를 Pandas Dataframe으로 합치기

In [33]:
import pandas as pd
weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc":descs
})
weather

,period,short_desc,temp,desc
0,Tonight,Mostly Clear,Low: 47 °F,"Tonight: Mostly clear, with a low around 47. W..."
1,Friday,Mostly Sunny,High: 63 °F,"Friday: Mostly sunny, with a high near 63. Wes..."
2,FridayNight,Mostly Cloudyand Breezythen ChanceRain,Low: 50 °F,Friday Night: A 50 percent chance of rain afte...
3,Saturday,Rain thenShowersLikely,High: 59 °F,"Saturday: Rain before noon, then showers likel..."
4,SaturdayNight,Mostly Clear,Low: 45 °F,"Saturday Night: Mostly clear, with a low aroun..."
5,Sunday,Sunny,High: 63 °F,"Sunday: Sunny, with a high near 63."
6,SundayNight,Mostly Clear,Low: 47 °F,"Sunday Night: Mostly clear, with a low around 47."
7,Monday,Sunny,High: 69 °F,"Monday: Sunny, with a high near 69."
8,MondayNight,Clear,Low: 53 °F,"Monday Night: Clear, with a low around 53."


In [54]:
temp_nums = weather["temp"].str.extract(r'(?P<digit>\d+)', expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    47
1    63
2    50
3    59
4    45
5    63
6    47
7    69
8    53
Name: digit, dtype: object

In [55]:
weather["temp_num"].mean()

55.111111111111114

In [56]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0     True
1    False
2     True
3    False
4     True
5    False
6     True
7    False
8     True
Name: temp, dtype: bool

In [57]:
weather[is_night]

,period,short_desc,temp,desc,temp_num,is_night
0,Tonight,Mostly Clear,Low: 47 °F,"Tonight: Mostly clear, with a low around 47. W...",47,True
2,FridayNight,Mostly Cloudyand Breezythen ChanceRain,Low: 50 °F,Friday Night: A 50 percent chance of rain afte...,50,True
4,SaturdayNight,Mostly Clear,Low: 45 °F,"Saturday Night: Mostly clear, with a low aroun...",45,True
6,SundayNight,Mostly Clear,Low: 47 °F,"Sunday Night: Mostly clear, with a low around 47.",47,True
8,MondayNight,Clear,Low: 53 °F,"Monday Night: Clear, with a low around 53.",53,True
